In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
class InductiveLPDataset(DisjointInductivePathDataset):
    """An inductive link prediction dataset for the ILPC 2022 Challenge."""

    
    
    
    def __init__(self , **kwargs):
        """Initialize the inductive link prediction dataset.

        :param size: "small" or "large"
        :param kwargs: keyword arguments to forward to the base dataset class, cf. DisjointInductivePathDataset
        """
        DATA_TYPE = "_fully_inductive.tsv"
        TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
        TEST_PATH = "MSCallGraph_test" + DATA_TYPE
        VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE
        INFERENCE_PATH = "MSCallGraph_inference" + DATA_TYPE


        super().__init__(
            transductive_training_path=os.getcwd()+"/"+TRAIN_PATH,
            inductive_inference_path=os.getcwd()+"/"+INFERENCE_PATH,
            inductive_validation_path=os.getcwd()+"/"+VALIDATE_PATH,
            inductive_testing_path=os.getcwd()+"/"+TEST_PATH,
            create_inverse_triples=True,
            eager=True,
            **kwargs
        )


In [3]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [4]:
dataset = InductiveLPDataset()

In [5]:
model_name = 'nodepiece_inductive'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
num_tokens = 20
embedding_dim = 200

In [8]:
model = InductiveNodePiece(
        triples_factory=dataset.transductive_training,
        inference_factory=dataset.inductive_inference,
        random_seed = seed,
        loss = loss,
        num_tokens = num_tokens,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

sampling:   0%|          | 0.00/9.06k [00:00<?, ?it/s]

No symbolic computation of output shape.


sampling:   0%|          | 0.00/3.79k [00:00<?, ?it/s]

No symbolic computation of output shape.


Number of parameters: 2600
Space occupied: 10400 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory nodepiece_inductive already exists.


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        mode=TRAINING,
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        mode=VALIDATION,
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        mode=TESTING,
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.inductive_inference,
    evaluation_triples_factory = dataset.inductive_validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.transductive_training,
        model=model,
        mode=TRAINING,  # must be specified for the inductive setup
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.transductive_training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 6.905938625335693
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.003296744595355643
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.02444209663995957
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.013869420617657608
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0025060756597667933
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.022589141502976418
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.012547609396278858
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0023134358885016494
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.021377431877952463
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.011845433883227055
Metric: validation.head.optimistic.hits_at_1 = 0.0006395906619763352
Metric: validation.tail.optimistic.hits_at_1 = 0.003837543971858011
Metric: validation.both.optimistic.hits_at_1 = 0

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 6.683731555938721
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.003583263534783715
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.027200169681249654
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.015391716608016685
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.002855785423889756
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.023055074736475945
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.012955429032444954
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.002662982024671611
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.02168035285610902
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.012171667440390315
Metric: validation.head.optimistic.hits_at_1 = 0.0009593859929645027
Metric: validation.tail.optimistic.hits_at_1 = 0.004157339302846178
Metric: validation.both.optimistic.hits_at_1 = 0.0

...

Step: 27
Metric: loss = 2.341808557510376
Step: 27
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.008007251890542133
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.01615620165659284
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.012081726773567487
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.007371579762548208
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.015611389651894571
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.011491484940052032
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0070755445437746586
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.015327733278388518
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.01120163891108159
Metric: validation.head.optimistic.hits_at_1 = 0.002878157978893508
Metric: validation.tail.optimistic.hits_at_1 = 0.003837543971858011
Metric: validation.both.optimistic.hits_at_1 = 0.

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 28
Metric: loss = 2.268432378768921
Step: 28
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.007938212023203767
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.016110671115757827
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.012024441569480794
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.00724763935431838
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.014742343686521055
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.010994992218911648
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0069327569105473935
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.01404620186659381
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0104894793885706
Metric: validation.head.optimistic.hits_at_1 = 0.002878157978893508
Metric: validation.tail.optimistic.hits_at_1 = 0.005756315957787016
Metric: validation.both.optimistic.hits_at_1 = 0.00

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 29
Metric: loss = 2.18854558467865
Step: 29
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.007952018229286768
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.015590580236486174
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.011771299232886475
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.007207181770354509
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.015097525902092457
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.011152353137731552
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.006854535762471723
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.014848099814061079
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.010851317788266401
Metric: validation.head.optimistic.hits_at_1 = 0.0031979533098816758
Metric: validation.tail.optimistic.hits_at_1 = 0.005116725295810681
Metric: validation.both.optimistic.hits_at_1 = 0

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
           0
0   6.905939
1   6.683732
2   6.496100
3   6.282094
4   6.070717
5   5.847056
6   5.642332
7   5.408726
8   5.214818
9   5.007322
10  4.780346
11  4.585206
12  4.391015
13  4.179098
14  4.003134
15  3.796273
16  3.629630
17  3.465006
18  3.293668
19  3.161256
20  3.026609
21  2.905393
22  2.776942
23  2.664813
24  2.552901
25  2.446806
26  2.341809
27  2.268432
28  2.188546


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.transductive_training.mapped_triples,
        additional_filter_triples=[
        dataset.transductive_training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/23.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.006286   0.004175     0.003709
hits_at_1                     0.003578   0.001831     0.001831
hits_at_3                     0.006346   0.003833     0.003152
hits_at_5                     0.007028   0.004600     0.004217
hits_at_10                    0.008007   0.006857     0.005239
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.008027   0.005668     0.004750
hits_at_1                     0.000128   0.000085     0.000085
hits_at_3                     0.000809   0.000596     0.000554
hits_at_5                     0.005878   0.002300     0.002300
hits_at_10                    0.010989   0.008007     0.007453
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.007156   0.004921     0.004230
hits_at_1                     0.001853   0.000958     0.000958
hits_at_3                     0.003578  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_validation.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/3.13k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.006119   0.005316     0.005013
hits_at_1                     0.001919   0.001919     0.001919
hits_at_3                     0.004157   0.004157     0.004157
hits_at_5                     0.007035   0.005117     0.005117
hits_at_10                    0.008954   0.007675     0.006396
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.026939   0.024863     0.023822
hits_at_1                     0.005437   0.005437     0.005437
hits_at_3                     0.020787   0.018228     0.017909
hits_at_5                     0.025903   0.025264     0.024944
hits_at_10                    0.073873   0.032939     0.032939
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.016529   0.015089     0.014417
hits_at_1                     0.003678   0.003678     0.003678
hits_at_3                     0.012472  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/3.98k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.007763   0.007098     0.006837
hits_at_1                     0.003769   0.003769     0.003769
hits_at_3                     0.006030   0.005779     0.005779
hits_at_5                     0.007789   0.006533     0.006281
hits_at_10                    0.008794   0.008291     0.008291
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.023096   0.021037     0.020078
hits_at_1                     0.004271   0.004271     0.004271
hits_at_3                     0.015578   0.014070     0.013819
hits_at_5                     0.022613   0.019849     0.019598
hits_at_10                    0.066332   0.027136     0.027136
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.015429   0.014067     0.013457
hits_at_1                     0.004020   0.004020     0.004020
hits_at_3                     0.010804  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['num_tokens'] = num_tokens
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                      num_tokens   
17                   embedding_dim   
18                   learning_rate   
19                       optimizer   
20                      num_epochs   
21                        patience   

                                                value  
0                                              cuda:0  
1            

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)